In [26]:
import os
from dotenv import load_dotenv

load_dotenv()

model_name = "gpt-4o-mini"
api_key=os.getenv("OPENAI_API_KEY")

In [42]:
import chromadb


chroma_db = chromadb.PersistentClient(path="db/chromadb")

In [43]:
from typing import List
from pydantic import BaseModel, Field


class SummaryDto(BaseModel):
    """Represents a summary with the title, content, and metadata."""

    title: str = Field(
        ..., description="Title of the summary", example="Rosneft announced dividend payment"
    )
    content: str = Field(
        ...,
        description="Main content of the summary, providing a concise overview",
        example="A robotic dog with a rabbit and a minigun is a new word in home security. A Chinese enthusiast has created an unusual hybrid, where a cute animal has become part of the combat system. Now this mecha-hare not only looks threatening, but also effectively performs its task, monitoring the yard.",
    )
    metadata: dict = Field(
        ...,
        description="Additional metadata related to the summary, such as the source",
        example={"source": "https://www.google.com"},
    )


In [44]:
summaries: List[SummaryDto] = [
    SummaryDto(
        title="Rosneft Announces Dividend Payment",
        content="Rosneft has declared a payment of dividends to its shareholders, highlighting its strong financial performance in the last quarter. The company's strategy to maintain stable earnings despite fluctuating oil prices has been praised by investors.",
        metadata={"source": "https://www.rosneft.com/press/releases/2023/rosneft-announces-dividend-payment"}
    ),
    SummaryDto(
        title="AI Revolutionizes Home Security with Robotic Dogs",
        content="A Chinese innovator has unveiled a robotic dog equipped with advanced surveillance capabilities and a combat system. This 'mecha-hare' is designed to offer both home security and a futuristic twist to traditional surveillance methods.",
        metadata={"source": "https://www.techcrunch.com/ai-robotic-dog-home-security"}
    ),
    SummaryDto(
        title="New Vaccination Guidelines Issued Amid COVID-19 Surge",
        content="The World Health Organization has updated its COVID-19 vaccination guidelines, recommending booster shots for high-risk groups. The new guidelines come in response to a spike in cases in several countries.",
        metadata={"source": "https://www.who.int/news-room/press-releases/2023/new-vaccination-guidelines"}
    ),
    SummaryDto(
        title="Tesla to Launch Full Self-Driving Cars by 2025",
        content="Tesla has announced plans to roll out fully autonomous vehicles by 2025. The new technology is expected to revolutionize the automotive industry and potentially reduce road accidents caused by human error.",
        metadata={"source": "https://www.cnbc.com/2023/09/tesla-full-self-driving-launch"}
    ),
    SummaryDto(
        title="NASA Discovers Water on Mars",
        content="NASA's latest mission has confirmed the presence of liquid water beneath the surface of Mars. This discovery is a significant step toward understanding the potential for life on the Red Planet.",
        metadata={"source": "https://www.nasa.gov/press-release/nasa-discovers-water-on-mars"}
    ),
    SummaryDto(
        title="Microsoft Acquires GitHub for $7.5 Billion",
        content="Microsoft has completed its acquisition of GitHub, a popular code hosting platform. This deal is part of Microsoft's strategy to strengthen its developer tools and cloud services offerings.",
        metadata={"source": "https://www.microsoft.com/en-us/news/press/2023/microsoft-acquires-github"}
    ),
    SummaryDto(
        title="Apple Introduces New AR Glasses at WWDC 2023",
        content="Apple has unveiled its latest product, AR Glasses, during the Worldwide Developers Conference (WWDC) 2023. The glasses are designed to enhance the augmented reality experience and are expected to be a game-changer in wearable technology.",
        metadata={"source": "https://www.apple.com/newsroom/2023/wwdc-2023-ar-glasses"}
    ),
    SummaryDto(
        title="Global Food Crisis Expected to Worsen in 2024",
        content="The United Nations has warned that the global food crisis is likely to worsen in 2024 due to ongoing conflicts, climate change, and supply chain disruptions. The organization is calling for urgent action to address the rising hunger levels.",
        metadata={"source": "https://www.un.org/en/food-crisis-2024-warning"}
    ),
    SummaryDto(
        title="Amazon Launches Drone Delivery Service",
        content="Amazon has officially launched its drone delivery service, allowing customers to receive packages within hours of ordering. The new service is expected to transform the logistics industry and reduce delivery times dramatically.",
        metadata={"source": "https://www.amazon.com/press-release/drone-delivery-launch"}
    ),
    SummaryDto(
        title="China's New Space Station to Host International Research",
        content="China's space station, Tiangong, is now open to international scientists for research opportunities. The station is expected to be a key player in advancing space exploration and scientific discovery.",
        metadata={"source": "https://www.space.com/china-new-space-station-research"}
    ),
]

In [45]:
import uuid
from llama_index.core import Document
from llama_index.core.schema import TextNode

bot_id = "12sdfsdfsdf123123213213"
text_nodes = []

for summary in summaries:
    text=f"{summary.title}\n\n{summary.content}"
    metadata = {"title": summary.title}
    metadata.update(summary.metadata)
    text_node = TextNode(text=text, metadata=metadata)
    text_nodes.append(text_node)
text_nodes

[TextNode(id_='054be7bd-e82a-4804-b377-144c6ef85f8c', embedding=None, metadata={'title': 'Rosneft Announces Dividend Payment', 'source': 'https://www.rosneft.com/press/releases/2023/rosneft-announces-dividend-payment'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="Rosneft Announces Dividend Payment\n\nRosneft has declared a payment of dividends to its shareholders, highlighting its strong financial performance in the last quarter. The company's strategy to maintain stable earnings despite fluctuating oil prices has been praised by investors.", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='4953a948-d739-4652-ba05-48ca07a24c04', embedding=None, metadata={'title': 'AI Revolutionizes Home Security with Robotic Dogs', 'source': 'https://www.techcrunch.com/ai-robotic-dog-home-security'}, 

In [46]:
from llama_index.core import StorageContext, VectorStoreIndex

chroma_db.delete_collection(bot_id)
vector_store = chroma_db.get_or_create_collection(bot_id)



NotFoundError: Collection [12sdfsdfsdf123123213213] does not exists

In [47]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [33]:
import nest_asyncio
nest_asyncio.apply()

In [48]:
index = VectorStoreIndex(text_nodes, storage_context)


## Addition summaries to save in vector index

In [35]:
more_summaries =[
    SummaryDto(
        title="Germany Unveils National Hydrogen Strategy",
        content="Germany has announced a comprehensive national hydrogen strategy aimed at reducing carbon emissions and transitioning to clean energy. The plan includes significant investments in hydrogen infrastructure and green technology development.",
        metadata={"source": "https://www.bmwi.de/EN/Topics/Energy/hydrogen-strategy.html"}
    ),
    SummaryDto(
        title="Google DeepMind Develops AI to Control Nuclear Fusion",
        content="DeepMind, an AI subsidiary of Google, has developed an artificial intelligence system capable of managing plasma in nuclear fusion reactors. This breakthrough could accelerate the development of sustainable and limitless energy sources.",
        metadata={"source": "https://deepmind.com/blog/article/AI-controls-nuclear-fusion"}
    ),
    SummaryDto(
        title="EU Implements Digital ID Wallet Across Member States",
        content="The European Union has begun rolling out its Digital Identity Wallet, enabling citizens to access public services, verify identity, and store official documents securely through a single digital application.",
        metadata={"source": "https://ec.europa.eu/commission/presscorner/detail/en/ip_23_1450"}
    ),
    SummaryDto(
        title="India's Chandrayaan-3 Successfully Lands on Moon",
        content="India's space agency, ISRO, has achieved a historic lunar landing with its Chandrayaan-3 mission. The spacecraft successfully touched down near the Moon's south pole, marking a significant milestone in space exploration.",
        metadata={"source": "https://www.isro.gov.in/Chandrayaan3_landing_success.html"}
    ),
    SummaryDto(
        title="Meta Introduces AI-Powered Virtual Reality Avatars",
        content="Meta has launched new AI-powered avatars for its virtual reality platforms. These avatars can mimic real-time facial expressions and voice tones, offering a more immersive experience in digital social environments.",
        metadata={"source": "https://about.fb.com/news/2023/meta-ai-vr-avatars-launch"}
    )
]

In [49]:
text_nodes_more = []

for summary in more_summaries:
    text=f"{summary.title}\n\n{summary.content}"
    metadata = {"title": summary.title}
    metadata.update(summary.metadata)
    text_node = TextNode(text=text, metadata=metadata)
    text_nodes_more.append(text_node)
text_nodes_more

[TextNode(id_='8de246aa-8cbd-4dc7-912d-78fa468f1f9f', embedding=None, metadata={'title': 'Germany Unveils National Hydrogen Strategy', 'source': 'https://www.bmwi.de/EN/Topics/Energy/hydrogen-strategy.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Germany Unveils National Hydrogen Strategy\n\nGermany has announced a comprehensive national hydrogen strategy aimed at reducing carbon emissions and transitioning to clean energy. The plan includes significant investments in hydrogen infrastructure and green technology development.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='965cb6ee-243c-43ea-b613-27654b2ef262', embedding=None, metadata={'title': 'Google DeepMind Develops AI to Control Nuclear Fusion', 'source': 'https://deepmind.com/blog/article/AI-controls-nuclear-fusion'}, e

In [54]:
collection = chroma_db.get_or_create_collection(bot_id)

In [56]:
from llama_index.vector_stores.chroma import ChromaVectorStore

update_vector_store = ChromaVectorStore(collection)

In [57]:
storage_context_updated = StorageContext.from_defaults(vector_store=update_vector_store)

In [58]:
index = VectorStoreIndex.from_vector_store(
    vector_store=update_vector_store,
    storage_context=storage_context_updated
)



In [60]:
index.insert_nodes(text_nodes_more)

In [61]:
query_engine = index.as_query_engine()
query_engine.query("What Germany has announced aimed at reducing carbon emissions")



Response(response='Germany has announced a comprehensive national hydrogen strategy aimed at reducing carbon emissions.', source_nodes=[NodeWithScore(node=TextNode(id_='8de246aa-8cbd-4dc7-912d-78fa468f1f9f', embedding=None, metadata={'title': 'Germany Unveils National Hydrogen Strategy', 'source': 'https://www.bmwi.de/EN/Topics/Energy/hydrogen-strategy.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Germany Unveils National Hydrogen Strategy\n\nGermany has announced a comprehensive national hydrogen strategy aimed at reducing carbon emissions and transitioning to clean energy. The plan includes significant investments in hydrogen infrastructure and green technology development.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.7572608717922922), NodeWithScore(node=TextNode(id_='48b79250

In [62]:
query_engine.query("When Tesla wants to Launch Full Self-Driving Cars?")

Response(response='Tesla plans to launch Full Self-Driving Cars.', source_nodes=[NodeWithScore(node=TextNode(id_='965cb6ee-243c-43ea-b613-27654b2ef262', embedding=None, metadata={'title': 'Google DeepMind Develops AI to Control Nuclear Fusion', 'source': 'https://deepmind.com/blog/article/AI-controls-nuclear-fusion'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Google DeepMind Develops AI to Control Nuclear Fusion\n\nDeepMind, an AI subsidiary of Google, has developed an artificial intelligence system capable of managing plasma in nuclear fusion reactors. This breakthrough could accelerate the development of sustainable and limitless energy sources.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.6474875611619653), NodeWithScore(node=TextNode(id_='1328551c-2cf8-4fe8-818c-3da3bc926bc2', em

In [35]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
    verbose=True
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
)




In [36]:
response = query_engine.query("What id doing Amazon with drone Services?")

In [63]:
collection3 = chroma_db.get_or_create_collection(bot_id)

In [64]:
vector_store3 = ChromaVectorStore(collection3)


In [65]:
storage_context_updated3 = StorageContext.from_defaults(vector_store=vector_store3)

In [66]:
index3 = VectorStoreIndex.from_vector_store(
    vector_store=vector_store3,
    storage_context=storage_context_updated3
)

In [67]:
query_engine3 = index3.as_query_engine()
query_engine3.query("When Tesla wants to Launch Full Self-Driving Cars?")

Response(response='Tesla has not been mentioned in the provided context information.', source_nodes=[NodeWithScore(node=TextNode(id_='965cb6ee-243c-43ea-b613-27654b2ef262', embedding=None, metadata={'title': 'Google DeepMind Develops AI to Control Nuclear Fusion', 'source': 'https://deepmind.com/blog/article/AI-controls-nuclear-fusion'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Google DeepMind Develops AI to Control Nuclear Fusion\n\nDeepMind, an AI subsidiary of Google, has developed an artificial intelligence system capable of managing plasma in nuclear fusion reactors. This breakthrough could accelerate the development of sustainable and limitless energy sources.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.6474875611619653), NodeWithScore(node=TextNode(id_='1328551c-2cf8-4fe8-81

In [70]:
clct = chroma_db.get_collection(bot_id)

NotFoundError: Collection [12sdfsdfsdf123123213213] does not exists

In [69]:
chroma_db.delete_collection(bot_id)


In [37]:
second_stream_summaries: List[SummaryDto] = [
    SummaryDto(
        title="Discovery of New Exoplanet in the Goldilocks Zone",
        content="Astronomers have discovered a new exoplanet located in the Goldilocks Zone of its star, where conditions are just right for liquid water to exist. The planet, named Kepler-452b, is 1.5 times the size of Earth and orbits a star similar to our Sun. This discovery raises hopes for finding potentially habitable planets beyond our solar system.",
        metadata={"source": "https://www.space.com/new-exoplanet-goldilocks-zone"}
    ),
    SummaryDto(
        title="Tesla Acquires SolarCity to Expand Clean Energy Portfolio",
        content="Tesla has completed its acquisition of SolarCity, a leading solar energy company, for $2.6 billion. This deal marks a significant step for Tesla in its efforts to integrate solar energy and storage solutions into its electric vehicle ecosystem. The acquisition will help Tesla expand its renewable energy services and reduce the reliance on fossil fuels.",
        metadata={"source": "https://www.bbc.com/tesla-acquires-solarcity"}
    ),
    SummaryDto(
        title="UN Holds Emergency Meeting to Address Climate Change Crisis",
        content="The United Nations convened an emergency meeting in response to the worsening climate change crisis, with global temperatures rising at an alarming rate. The meeting focused on immediate actions needed to reduce carbon emissions, including a shift towards renewable energy sources and the implementation of stricter environmental regulations. Countries around the world are expected to commit to more ambitious climate goals in the coming years.",
        metadata={"source": "https://www.un.org/climate-change-emergency-meeting"}
    ),
    SummaryDto(
        title="Tesla to Launch Full Self-Driving Cars by 2025",
        content="Tesla has announced plans to roll out fully autonomous vehicles by 2025. The new technology is expected to revolutionize the automotive industry and potentially reduce road accidents caused by human error.",
        metadata={"source": "https://www.cnbc.com/2023/09/tesla-full-self-driving-launch"}
    ),
    SummaryDto(
        title="NASA Discovers Water on Mars",
        content="NASA's latest mission has confirmed the presence of liquid water beneath the surface of Mars. This discovery is a significant step toward understanding the potential for life on the Red Planet.",
        metadata={"source": "https://www.nasa.gov/press-release/nasa-discovers-water-on-mars"}
    ),
    SummaryDto(
        title="Microsoft Acquires GitHub for $7.5 Billion",
        content="Microsoft has completed its acquisition of GitHub, a popular code hosting platform. This deal is part of Microsoft's strategy to strengthen its developer tools and cloud services offerings.",
        metadata={"source": "https://www.microsoft.com/en-us/news/press/2023/microsoft-acquires-github"}
    ),
    SummaryDto(
        title="Apple Introduces New AR Glasses at WWDC 2023",
        content="Apple has unveiled its latest product, AR Glasses, during the Worldwide Developers Conference (WWDC) 2023. The glasses are designed to enhance the augmented reality experience and are expected to be a game-changer in wearable technology.",
        metadata={"source": "https://www.apple.com/newsroom/2023/wwdc-2023-ar-glasses"}
    ),
    SummaryDto(
        title="Global Food Crisis Expected to Worsen in 2024",
        content="The United Nations has warned that the global food crisis is likely to worsen in 2024 due to ongoing conflicts, climate change, and supply chain disruptions. The organization is calling for urgent action to address the rising hunger levels.",
        metadata={"source": "https://www.un.org/en/food-crisis-2024-warning"}
    ),
    SummaryDto(
        title="Amazon Launches Drone Delivery Service",
        content="Amazon has officially launched its drone delivery service, allowing customers to receive packages within hours of ordering. The new service is expected to transform the logistics industry and reduce delivery times dramatically.",
        metadata={"source": "https://www.amazon.com/press-release/drone-delivery-launch"}
    ),
    SummaryDto(
        title="China's New Space Station to Host International Research",
        content="China's space station, Tiangong, is now open to international scientists for research opportunities. The station is expected to be a key player in advancing space exploration and scientific discovery.",
        metadata={"source": "https://www.space.com/china-new-space-station-research"}
    ),
]

In [38]:
from llama_index.core.tools import QueryEngineTool


def create_engine_tool(summary: SummaryDto) -> QueryEngineTool:
    """
    Create a query engine tool for a summary.
    """
    return QueryEngineTool.from_defaults(
        query_engine=query_engine, description=f"Provides information about possible existing summaries in db about - {summary.title}.")


In [39]:
system_prompt = """
You are a specialized AI agent tasked with sanitizing user notifications to avoid sending duplicate or redundant information.
Your role is to process new summaries of notifications and remove content that has already been sent to the user, using a semantic search tool.

Follow these rules:

1. For each summary:
    - Use the query engine tools to retrieve past notifications that are semantically similar.
    - Compare the new summary with the retrieved content.
    - Remove or rewrite any parts of the new summary that duplicate information in retrieved content.
    - Preserve only novel and valuable content.

2. Tool usage:
    - Always use the corresponding query engine tool for summary topic.
    - The tool will return past notifications that the user had already received.

3. Ensure the sanitized summary is concise, clear, and informative.
4. Do not fabricate new information — preserve the original intent.

5. General rules:
    - Maintain original tone and structure where possible.
    - Avoid repeating facts or messages already present in past notifications.
    - Only return cleaned summaries — do not include analysis or reasoning in the final output.

### **Output Format**:
Your response should be a list of summaries, where each summary is represented as a dictionary with the following fields:
```json
[
  {
    "title": "Title of the Combined Summary",
    "content": "The full, rewritten text of the combined summary",
    "metadata": { 
      "key1": "value1", 
      "key2": "value2" 
    }
  },
  ...
]

**Dont add any explanation/description by yourself in the output.**

CHECK YOUR ANSWER BEFORE RETURNING IT!!!.
"""


In [40]:
import json
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI


tools = [
    create_engine_tool(summary)
    for summary in second_stream_summaries
]




In [41]:
llm = OpenAI(model=model_name, api_key=api_key)
agent = FunctionAgent(
    llm=llm,
    tools=tools,
    system_prompt=system_prompt
)

In [42]:
summaries_json = json.dumps([summary.model_dump() for summary in second_stream_summaries])

summaries_json


'[{"title": "Discovery of New Exoplanet in the Goldilocks Zone", "content": "Astronomers have discovered a new exoplanet located in the Goldilocks Zone of its star, where conditions are just right for liquid water to exist. The planet, named Kepler-452b, is 1.5 times the size of Earth and orbits a star similar to our Sun. This discovery raises hopes for finding potentially habitable planets beyond our solar system.", "metadata": {"source": "https://www.space.com/new-exoplanet-goldilocks-zone"}}, {"title": "Tesla Acquires SolarCity to Expand Clean Energy Portfolio", "content": "Tesla has completed its acquisition of SolarCity, a leading solar energy company, for $2.6 billion. This deal marks a significant step for Tesla in its efforts to integrate solar energy and storage solutions into its electric vehicle ecosystem. The acquisition will help Tesla expand its renewable energy services and reduce the reliance on fossil fuels.", "metadata": {"source": "https://www.bbc.com/tesla-acquires-

In [43]:
handler = agent.run(summaries_json)


In [44]:
from llama_index.core.agent.workflow import AgentStream
from llama_index.core.agent.workflow import AgentOutput, ToolCallResult, ToolCall

result = None
async for event in handler.stream_events():
    if (hasattr(event, "current_agent_name")):
        print(f"agent: {event.current_agent_name}")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
            result = event.response.content
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")

agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent
agent: Agent

In [45]:
result2 = await agent.run(summaries_json)

In [51]:
json_parsed = json.loads(result2.response.content)
json_parsed

[{'title': 'Discovery of New Exoplanet in the Goldilocks Zone',
  'content': 'Astronomers have discovered a new exoplanet named Kepler-452b, located in the Goldilocks Zone of its star, where conditions are just right for liquid water to exist. This discovery raises hopes for finding potentially habitable planets beyond our solar system.',
  'metadata': {'source': 'https://www.space.com/new-exoplanet-goldilocks-zone'}},
 {'title': 'Tesla Acquires SolarCity to Expand Clean Energy Portfolio',
  'content': 'Tesla has completed its acquisition of SolarCity, a leading solar energy company, for $2.6 billion. This deal marks a significant step for Tesla in its efforts to integrate solar energy and storage solutions into its electric vehicle ecosystem.',
  'metadata': {'source': 'https://www.bbc.com/tesla-acquires-solarcity'}},
 {'title': 'UN Holds Emergency Meeting to Address Climate Change Crisis',
  'content': 'The United Nations convened an emergency meeting in response to the worsening cli

In [47]:
json_parsed = json.loads(result2.response.content)
summaries = [SummaryDto(title=summary["title"], content=summary["content"], metadata=summary["metadata"]) for summary in json_parsed]
print(summaries)


[SummaryDto(title='Discovery of New Exoplanet in the Goldilocks Zone', content='Astronomers have discovered a new exoplanet named Kepler-452b, located in the Goldilocks Zone of its star, where conditions are just right for liquid water to exist. This discovery raises hopes for finding potentially habitable planets beyond our solar system.', metadata={'source': 'https://www.space.com/new-exoplanet-goldilocks-zone'}), SummaryDto(title='Tesla Acquires SolarCity to Expand Clean Energy Portfolio', content='Tesla has completed its acquisition of SolarCity, a leading solar energy company, for $2.6 billion. This deal marks a significant step for Tesla in its efforts to integrate solar energy and storage solutions into its electric vehicle ecosystem.', metadata={'source': 'https://www.bbc.com/tesla-acquires-solarcity'}), SummaryDto(title='UN Holds Emergency Meeting to Address Climate Change Crisis', content='The United Nations convened an emergency meeting in response to the worsening climate c

In [48]:
for summary in summaries:
    print(summary.title)
    print(summary.content)
    print(summary.metadata)
    print("--------------------------------")



Discovery of New Exoplanet in the Goldilocks Zone
Astronomers have discovered a new exoplanet named Kepler-452b, located in the Goldilocks Zone of its star, where conditions are just right for liquid water to exist. This discovery raises hopes for finding potentially habitable planets beyond our solar system.
{'source': 'https://www.space.com/new-exoplanet-goldilocks-zone'}
--------------------------------
Tesla Acquires SolarCity to Expand Clean Energy Portfolio
Tesla has completed its acquisition of SolarCity, a leading solar energy company, for $2.6 billion. This deal marks a significant step for Tesla in its efforts to integrate solar energy and storage solutions into its electric vehicle ecosystem.
{'source': 'https://www.bbc.com/tesla-acquires-solarcity'}
--------------------------------
UN Holds Emergency Meeting to Address Climate Change Crisis
The United Nations convened an emergency meeting in response to the worsening climate change crisis, focusing on immediate actions need